In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from alpha_vantage.timeseries import TimeSeries
import plotly.graph_objects as go

import warnings as wa
wa.filterwarnings('ignore')

In [2]:
class Data:

    def __init__(self):
        self.data = {}
        self.tskey = TimeSeries(key='VC612M8YBUMNGVZ2')
        
    def fetchData(self, sym):
        df, meta_data = self.tskey.get_intraday(sym)
        self.data[f'{sym}'] = df

        return df , sym

    def convertData(self,sym):
        df = self.data[sym]
        df = pd.DataFrame(df).transpose().reset_index()
        df.columns = ['timestamp','open','high','low','close','volume']

        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df['volume'] = pd.to_numeric(df['volume'])
        df['close'] = pd.to_numeric(df['close'])
        df['high'] = pd.to_numeric(df['high'])
        df['low'] = pd.to_numeric(df['low'])
        df['open'] = pd.to_numeric(df['open'])
        self.data[f'{sym}'] = df

        return df

    def __getitem__(self, key):
        return self.data[key]
    

def indicator1(df , time):
    df['indicator'] = np.nan
    for i in range(time-1, len(df)):
        df['indicator'][i] = sum(df['close'][i-time+1 : i+1])/time

    return df[['timestamp','indicator']].copy()




In [3]:
script = Data()
script.fetchData('NVDA')

({'2023-02-14 20:00:00': {'1. open': '228.6300',
   '2. high': '228.6400',
   '3. low': '228.3000',
   '4. close': '228.4300',
   '5. volume': '31171'},
  '2023-02-14 19:45:00': {'1. open': '228.6500',
   '2. high': '228.7000',
   '3. low': '228.5000',
   '4. close': '228.5000',
   '5. volume': '15535'},
  '2023-02-14 19:30:00': {'1. open': '228.5000',
   '2. high': '228.6800',
   '3. low': '228.2000',
   '4. close': '228.6500',
   '5. volume': '18434'},
  '2023-02-14 19:15:00': {'1. open': '228.5000',
   '2. high': '228.5000',
   '3. low': '228.0600',
   '4. close': '228.5000',
   '5. volume': '15091'},
  '2023-02-14 19:00:00': {'1. open': '228.5200',
   '2. high': '228.6000',
   '3. low': '228.3600',
   '4. close': '228.5000',
   '5. volume': '8792'},
  '2023-02-14 18:45:00': {'1. open': '228.9300',
   '2. high': '228.9300',
   '3. low': '228.4900',
   '4. close': '228.5500',
   '5. volume': '24262'},
  '2023-02-14 18:30:00': {'1. open': '229.1700',
   '2. high': '229.1800',
   '3. l

In [4]:
df = script.convertData('NVDA')
df

,timestamp,open,high,low,close,volume
0,2023-02-14 20:00:00,228.63,228.6400,228.3000,228.4300,31171
1,2023-02-14 19:45:00,228.65,228.7000,228.5000,228.5000,15535
2,2023-02-14 19:30:00,228.50,228.6800,228.2000,228.6500,18434
3,2023-02-14 19:15:00,228.50,228.5000,228.0600,228.5000,15091
4,2023-02-14 19:00:00,228.52,228.6000,228.3600,228.5000,8792
...,...,...,...,...,...,...
95,2023-02-13 12:15:00,219.03,219.3200,218.2700,218.7800,1208998
96,2023-02-13 12:00:00,219.40,219.7990,218.3400,219.0600,1534477
97,2023-02-13 11:45:00,218.95,219.6731,217.8300,219.4013,1786884
98,2023-02-13 11:30:00,218.00,219.0841,217.9900,218.9265,2201363


In [5]:
indicator1(df, time = 5)

,timestamp,indicator
0,2023-02-14 20:00:00,NaN
1,2023-02-14 19:45:00,NaN
2,2023-02-14 19:30:00,NaN
3,2023-02-14 19:15:00,NaN
4,2023-02-14 19:00:00,228.51600
...,...,...
95,2023-02-13 12:15:00,218.99644
96,2023-02-13 12:00:00,218.98700
97,2023-02-13 11:45:00,219.09626
98,2023-02-13 11:30:00,219.03356


In [6]:
class Strategy:

    def __init__(self , symbol):
        self.symbol  = symbol
        self.df = pd.DataFrame()

    def scriptData(self):
        self.script = Data()
        self.script.fetchData(self.symbol)
        self.script.convertData(self.symbol)

        self.df = self.script.data[self.symbol]

        self.indicator = indicator1(df , time=5)

        self.df['indicator_data'] = self.indicator['indicator']

    def signals(self):
        df = self.df
        df['signal'] = np.nan
        for i in range(len(df)):
            if df.iloc[i-1]['close']>df.iloc[i-1]['indicator_data'] and df.iloc[i]['close'] < df.iloc[i]['indicator_data']:
                df['signal'][i] = 'BUY'
            elif df.iloc[i-1]['close'] < df.iloc[i-1]['indicator_data'] and df.iloc[i]['close'] > df.iloc[i]['indicator_data']:
                df['signal'][i] = 'SELL'

        self.df = df
        return self.df[['timestamp','signal']].copy().dropna()

    def plot(self):
        df = self.df
        fig = px.line(df , y = ['close','indicator_data'])
        fig.show()

        fig = go.Figure(data=[go.Candlestick(
                       open=df['open'], high=df['high'],
                       low=df['low'], close=df['close'] ) ])
        fig.add_trace(go.Line(y=df['indicator_data']))
        fig.show()

In [7]:
strategy = Strategy('NVDA')
strategy.scriptData()
strategy.signals()

,timestamp,signal
5,2023-02-14 18:45:00,SELL
9,2023-02-14 17:45:00,BUY
14,2023-02-14 16:30:00,SELL
18,2023-02-14 15:30:00,BUY
21,2023-02-14 14:45:00,SELL
22,2023-02-14 14:30:00,BUY
26,2023-02-14 13:30:00,SELL
28,2023-02-14 13:00:00,BUY
35,2023-02-14 11:15:00,SELL
36,2023-02-14 11:00:00,BUY
